분석에 이용한 자료는 2020-03-11 22:00에 수집하였습니다.

In [185]:
import pickle
import json
import requests
import pandas as pd
import numpy as np
import chart_studio

import plotly
import plotly.graph_objects as go

from pandas import DataFrame
from scipy import stats

In [144]:
plotly.offline.init_notebook_mode(connected=True)

In [148]:
with open('data.pickle', 'rb') as f:
    fifa = pickle.load(f)

In [173]:
spId_url = requests.get('https://static.api.nexon.co.kr/fifaonline4/latest/spid.json')
spId_parsed_data = spId_url.json()
spId = pd.DataFrame(spId_parsed_data)
spId

,id,name
0,101000001,데이비드 시먼
1,101000195,로비 파울러
2,101000393,테디 셰링엄
3,101000805,옌스 레만
4,101001040,호베르투 카를루스
5,101001075,A. 델피에로
6,101001114,로베르토 바조
7,101001183,F. 칸나바로
8,101001201,잔프랑코 졸라
9,101001397,지네딘 지단


In [146]:
spposition_url = requests.get('https://static.api.nexon.co.kr/fifaonline4/latest/spposition.json')
spposition_parsed_data = spposition_url.json()
spposition = pd.DataFrame(spposition_parsed_data)

- ```assist``` : 평균 어시스트 수
- ```blcok``` : 평균 블락 성공 수
- ```dribble``` : 평균 드리블 거리(야드)
- ```effectiveShoot``` : 평균 유효 슛 수
- ```goal``` : 평균 득점 수
- ```matchCount``` : 해당 포지션으로 경기 참여한 횟수
- ```passSuccess``` : 평균 패스 성공 수
- ```tackle``` : 평균 태클 성공 수

In [149]:
fifa.head()

,spId,spPosition,assist,block,dribble,effectiveShoot,goal,matchCount,tackle,passRate
0,101000001,0,0.0,0.0,0.0,0.0,0.0,4,0.0,0.928571
1,101000001,28,0.0,0.0,0.0,0.0,0.0,2,0.0,NaN
2,101000195,4,0.0,0.0,0.0,0.0,0.0,1,0.0,1.000000
3,101000195,12,0.5,0.0,85.0,2.0,0.0,2,0.0,0.848485
4,101000195,14,2.0,0.0,0.0,0.0,0.0,1,0.0,0.928571


### 골키퍼(GK)

In [159]:
fifa[(fifa.spPosition == 0) & (fifa.matchCount == 20)].describe()

,spId,spPosition,assist,block,dribble,effectiveShoot,goal,matchCount,tackle,passRate
count,3.240000e+02,324.0,324.000000,324.0,324.000000,324.000000,324.000000,324.0,324.000000,324.000000
mean,3.243206e+08,0.0,0.000309,0.0,0.095525,0.002160,0.001235,20.0,0.004321,0.871794
std,1.261075e+08,0.0,0.003922,0.0,1.517760,0.011603,0.007771,0.0,0.015634,0.063646
min,2.010005e+08,0.0,0.000000,0.0,0.000000,0.000000,0.000000,20.0,0.000000,0.652174
25%,2.141924e+08,0.0,0.000000,0.0,0.000000,0.000000,0.000000,20.0,0.000000,0.841270
50%,3.001633e+08,0.0,0.000000,0.0,0.000000,0.000000,0.000000,20.0,0.000000,0.879655
75%,5.011691e+08,0.0,0.000000,0.0,0.000000,0.000000,0.000000,20.0,0.000000,0.916667
max,5.022346e+08,0.0,0.050000,0.0,27.250000,0.100000,0.050000,20.0,0.100000,0.984375


골키퍼의 스탯 중 ```passRate```의 표준편차가 가장 크다. (```dribble```의 경우 이상치로 간주)

In [172]:
gk = fifa[(fifa.spPosition == 0) & (fifa.matchCount == 20)]
gk_top10 = gk.sort_values(by = 'passRate', ascending = False).head(10)
gk_top10

,spId,spPosition,assist,block,dribble,effectiveShoot,goal,matchCount,tackle,passRate
43629,218199987,0,0.0,0.0,0.0,0.0,0.0,20,0.00,0.984375
34174,215215223,0,0.0,0.0,0.0,0.0,0.0,20,0.00,0.981481
123009,502199987,0,0.0,0.0,0.0,0.0,0.0,20,0.00,0.977273
61855,300190778,0,0.0,0.0,0.0,0.0,0.0,20,0.00,0.977273
108934,501173373,0,0.0,0.0,0.0,0.0,0.0,20,0.00,0.977273
123701,502203004,0,0.0,0.0,0.0,0.0,0.0,20,0.00,0.975610
110146,501183497,0,0.0,0.0,0.0,0.0,0.0,20,0.00,0.975000
40570,217169181,0,0.0,0.0,0.0,0.0,0.0,20,0.00,0.969231
81995,300226753,0,0.0,0.0,0.0,0.0,0.0,20,0.05,0.968750
119554,501236583,0,0.0,0.0,0.0,0.0,0.0,20,0.00,0.967742


패스 성공률 상위 10선수의 패스 성공률

In [198]:
gk_passRate = list(gk_top10['passRate'])

패스 성공률 상위 10선수의 이름

In [230]:
gk_topId = []

for i in gk_top10['spId'] :
    gk_topId.append(i)

gk_topSp = []

for j in gk_topId :
    gk_topSp.append(list(spId[spId['id'] == j].name))

gk_topSp = sum(gk_topSp, [])
gk_topSp

['J. 실러선',
 'W. 베니테스',
 'J. 실러선',
 '예룬 주트',
 'S. 로메로',
 '양한빈',
 'O. 카르네지스',
 'K. 베르메이르',
 '앙드레 오나나',
 'J. 파블렌카']

In [227]:
fig = go.Figure([go.Bar(x=gk_topSp, y=gk_passRate)])
fig.show()

오각형 모양의 선수 스탯 그래프 그리기 연습

In [231]:
data = [go.Scatterpolar(
    r = [test_gk['assist'].values[0],test_gk['block'].values[0], test_gk['effectiveShoot'].values[0],
         test_gk['passRate'].values[0],test_gk['tackle'].values[0]],
    
    theta = ['assist', 'block', 'effectiveShoot', 'passRate', 'tackle'],

    fill = 'toself')]


fig = go.Figure(data = data)
fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=False
)

fig.show()